## ECHO counter-algorithm
EPA uses a "sophisticated algorithm" to determine whether facilities are in compliance with the Clean Water Act, based on the discharges they report. But are facilities "gaming" these to avoid finding themselves in the most strict category of non-compliance, "Significant Non-Compliance (SNC)"?

Background, from ECHO's data dictionary (https://echo.epa.gov/help/reports/dfr-data-dictionary#compbyqtrunder "CWA/NPDES Compliance Status"): "EPA's data system uses a sophisticated algorithm looking at severity and duration of violations when determining whether a facility is considered to be in Significant/Category I Noncompliance (SNC)." 

Some relevant detail: "Violation of Monthly Average Effluent Limits—Data that exceeds or equals the limit times the Technical Review Criteria (TRC) for 2 months during a 6-month period, where the TRC is 1.4 for Group I pollutants and 1.2 for Group II pollutants (Appendix A to 40 CFR Part 123 contains a list of Group I and II pollutants); and data that exceeds the limit for 4 months during a 6-month period."

An example: let's say we're talking about a group I pollutant like oil and grease. If you're a facility discharging oil and grease and you were at >= 140% (100% limit x 1.4 TRC) for 2 months, you'd trigger SNC. Also if you were >=100% for 4 months out of 6, you'd trigger SNC. So, a facility might do something like this: M1-1000%+ M2-139% M3-139% M4-99% M5-99% M6-99% The facility could get away with a month of big, 1000%+ discharging (not withstanding the "single event" regulation that would itself trigger SNC). Then it could get away with two more months of 100%+ discharging. But in the fourth month, it'd have to drop down to below 100%/the limit to avoid the second part of the rule (can't exceed the limit for 4 months out of 6). 

This notebook checks to see if a facility was at just 3 months over the limit during a 6 month/2 quarters period (avoiding the second part of the Violation of Monthly Average Effluent Limits rule) AND was also only at 1.4 x the limit for one month (avoiding the first part of the rule).

---

### First, we load some libraries

In [46]:
import requests
import time
import csv
import json
from statistics import *
from collections import Counter
import datetime

import urllib.parse
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import folium

#### Then, we get ready to call up the ECHO database

In [11]:
f = [] # We will save inspection data by facility name in this list

#### Now, we're going to call up the ECHO database to try to get Clean Water Act compliance data
We'll get the 3 year compliance history for each major source of water pollution in our state.

In [ ]:
sql = "select REGIS, EXCEEDENCE_PCT, MONITORING_PERIOD_END_DATE, PARAMETER_DESC from NPDES_EFF_VIOLATIONS where NPDES_ID like '" + my_state + "%'" #+ \


In [103]:
# Set up our geography. By default, we'll look at states. But we can instead choose an EPA region (e.g. 04), zip codes, counties, city, or more...
zipc = 'p_zip='+'53703'
reg= 'p_reg=' + '04'
state= 'p_st=' + 'WI'
geo=state # By default, we'll look at states.
my_state="ME"
# call up ECHO API
#url="https://ofmpub.epa.gov/echo/echo_rest_services.get_facility_info?output=json&"+geo+"&p_act=Y&p_maj=Y&p_med=W"
#print("...Getting the data...")
#contents = requests.get(url)
#obj = contents.json() # Get the results from ECHO and turn them into something Python can work with

#facilities = obj['Results']['Facilities'] # Grab the information on the facilities

#for facility in facilities:
#    f.append(facility['RegistryID']) #store facility information in our list
    # Go to the next facility...

#print("DONE!")

# Call up ECHO DB
sql = "select NPDES_ID, EXCEEDENCE_PCT, MONITORING_PERIOD_END_DATE, PARAMETER_DESC" + \
    " from NPDES_EFF_VIOLATIONS where  NPDES_ID like '" + my_state + "%' and" +\
    " MONITORING_PERIOD_END_DATE like '%2020%' or" + \
    " MONITORING_PERIOD_END_DATE like '%2019%'" #+\
     #my_state not working
    #" FAC_INFORMAL_COUNT, FAC_FORMAL_ACTION_COUNT," + \
    #" FAC_DAYS_LAST_INSPECTION, FAC_DATE_LAST_INSPECTION, FAC_INSPECTION_COUNT" + \
    #" from ECHO_EXPORTER where FAC_MAJOR_FLAG = 'Y'" #where \
    #" FAC_EPA_REGION  = 3 "
url='http://apps.tlt.stonybrook.edu/echoepa/?query='
data_location=url+urllib.parse.quote(sql)
print(data_location)
data = pd.read_csv(data_location,encoding='iso-8859-1',header = 0)
#data.set_index("NPDES_ID", inplace=True)
data

http://apps.tlt.stonybrook.edu/echoepa/?query=select%20NPDES_ID%2C%20EXCEEDENCE_PCT%2C%20MONITORING_PERIOD_END_DATE%2C%20PARAMETER_DESC%20from%20NPDES_EFF_VIOLATIONS%20where%20%20NPDES_ID%20like%20%27ME%25%27%20and%20MONITORING_PERIOD_END_DATE%20like%20%27%252020%25%27%20or%20MONITORING_PERIOD_END_DATE%20like%20%27%252019%25%27


,NPDES_ID,EXCEEDENCE_PCT,MONITORING_PERIOD_END_DATE,PARAMETER_DESC
0,MS0020940,NaN,04/30/2019,"Oxygen, dissolved [DO]"
1,MAG250979,NaN,04/30/2019,pH
2,KYGE40788,NaN,08/31/2019,"Flow, in conduit or thru treatment plant"
3,KYGE40549,NaN,03/31/2019,"Iron, total recoverable"
4,LA0039829,NaN,07/31/2019,"Flow, in conduit or thru treatment plant"
5,NE0030724,NaN,09/30/2019,"Solids, total suspended"
6,NC0020486,NaN,01/31/2019,"Chlorine, total residual"
7,NE0030724,NaN,05/31/2019,"Nitrogen, total [as N]"
8,KYGE40844,NaN,08/31/2019,Specific conductance
9,IL0044661,NaN,06/30/2019,pH


In [1]:
data["PARAMETER_DESC"].unique()
data['MONITORING_PERIOD_END_DATE'] = pd.to_datetime(data['MONITORING_PERIOD_END_DATE'], format="%m/%d/%Y")
month=data.groupby(['MONITORING_PERIOD_END_DATE','NPDES_ID']).count()
#month.resample('M').sum()

facilities = data.groupby(['NPDES_ID', 'PARAMETER_DESC']).count()
facilities["Percent"] = (facilities["EXCEEDENCE_PCT"]/facilities['MONITORING_PERIOD_END_DATE'])*100
facilities = facilities.rename(columns={"PARAMETER_DESC": "Pollutant", "EXCEEDENCE_PCT": "Times exceeded", "MONITORING_PERIOD_END_DATE": "Across how many monitoring periods"})
#facilities
#badactors - always above the limit
badactors = facilities[(facilities["Percent"] == 100)]
badactors

fac = data.groupby(['NPDES_ID', 'MONITORING_PERIOD_END_DATE']).count()
fac
#data[data["NPDES_ID"]=="AK0053384"]
#data[data["NPDES_ID"]=="AK0028525"]

NameError: name 'data' is not defined

### How many facilities are we talking about?

In [26]:
print("There are "+str(len(f))+" facilities in this state")

There are 342 facilities in this state


#### Now that we have information on each facility...
...we're going to drill down into it and look at reported discharges and whether the facility was in or out of compliance as a result. 

This may take a while if run across every EPA facility in our state.

In [42]:
flagged=[] # We'll store facilities that appear to be "gaming" EPA's Significant Non-Compliance algorithm here in this list

# Look up each facility's CWA monthly violations
for fid in f: # For each facility...
    
    url="https://ofmpub.epa.gov/echo/dfr_rest_services.get_cwa_eff_compliance?output=JSON&p_id="+fid # call up ECHO to get specifics about compliance
    contents = requests.get(url, time.sleep(10)) # Try to avoid looking like a robot by waiting 10 seconds between queries
    obj = contents.json() # Get the results from ECHO and turn them into something Python can work with
    print(url)
    
    try:
        if obj['Results']["CWAEffluentCompliance"]["Sources"]:
            for s in obj['Results']["CWAEffluentCompliance"]["Sources"]: # For each of the facility's sources of water pollution....Not sure why there would be multiple sources
                for p in s['Parameters']: # And for each pollutant (or, parameter) discharged from that source...
                    
                    print(fid + ": " + p['ParameterName'])
                    nums=[] # We'll store the % over permitted limits here in this list...
                    statuses = "" # We'll temporarily store each facility's status (violation or no) here as a big string
                    
                    x=1
                    while x<40: # Loop through 39 months of data
                        code="Mnth"+str(x)+"Value"
                        if p[code] is not None and "%" in p[code]: # Based on how ECHO data is structured, we need to find where there is a % over limit indicated
                            statuses = statuses + "V" # If over the limit, indicate a violation
                            # Find the % of the limit that was exceeded by extracting the number from the string
                            pindex=p[code].find("%")
                            sindex=p[code].find(" ")
                            num=int(p[code][sindex:pindex])
                            nums.append(num)
                            print("Violation: " + str(num) + "% above limit")
                        else: # In this month and for this pollutant, the facility did not exceed permit limit
                            statuses = statuses + "C"
                            num = 0
                            nums.append(num)
                            print("in Compliance")
                        if x % 3 == 0 and x >= 6: # Check for suspicious patterns over last six months
                            # The pattern we're looking for: only 3 months above the permitted limit - avoiding 4 month rule
                            xindex = x - 6
                            count = len([i for i in nums[xindex:x] if i > 0]) # Number of times facility was over the limit
                            TRCcount = len([i for i in nums[xindex:x] if i > 39]) # Number of times facility was over the 1.4 TRC threshold. #TO DO: change from 39 depending on pollutant group.
                            if count == 3 and TRCcount <= 1:
                                print("suspicious? just 3 months above limit, only one or less of which was above TRC")
                                # Count how many times this suspicious pattern occurs for this facility/pollutant....
                                flagged.append({"fac": fid, "pollutant": p['ParameterName'], "range": [xindex, x]})
                        x+=1
                        
                    print(statuses)
                    
    except:
        print("Couldn't get this facility's info!")
        pass
    
print("DONE!")

https://ofmpub.epa.gov/echo/dfr_rest_services.get_cwa_eff_compliance?output=JSON&p_id=110006618589
110006618589: BOD, 5-day, percent removal
in Compliance
in Compliance
Violation: 20% above limit
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
Violation: 95% above limit
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
CCVCCCCCCCCCCCCCCCCCCCCCCCCVCCCCCCCCCCC
110006618589: Chlorine, total residual
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Complianc

https://ofmpub.epa.gov/echo/dfr_rest_services.get_cwa_eff_compliance?output=JSON&p_id=110000605934
Couldn't get this facility's info!
https://ofmpub.epa.gov/echo/dfr_rest_services.get_cwa_eff_compliance?output=JSON&p_id=110000417228
Couldn't get this facility's info!
https://ofmpub.epa.gov/echo/dfr_rest_services.get_cwa_eff_compliance?output=JSON&p_id=110038166365
110038166365: Copper, total recoverable
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
Violation: 42% above limit
Violation: 177% above limit
Violation: 54% above limit
Violation: 3% above limit
in Compliance
in Compliance
in Compliance
in Compliance
Violation: 7% above limit
Violation: 19% above limit
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Complianc

https://ofmpub.epa.gov/echo/dfr_rest_services.get_cwa_eff_compliance?output=JSON&p_id=110005427323
Couldn't get this facility's info!
https://ofmpub.epa.gov/echo/dfr_rest_services.get_cwa_eff_compliance?output=JSON&p_id=110009515417
Couldn't get this facility's info!
https://ofmpub.epa.gov/echo/dfr_rest_services.get_cwa_eff_compliance?output=JSON&p_id=110002050244
110002050244: Chlorine, total residual
in Compliance
in Compliance
in Compliance
in Compliance
Violation: 153% above limit
in Compliance
in Compliance
Violation: 12% above limit
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
CCCCVCCVCCCCCCCCCCCCC

https://ofmpub.epa.gov/echo/dfr_rest_services.get_cwa_eff_compliance?output=JSON&p_id=110064253196
Couldn't get this facility's info!
https://ofmpub.epa.gov/echo/dfr_rest_services.get_cwa_eff_compliance?output=JSON&p_id=110000422141
Couldn't get this facility's info!
https://ofmpub.epa.gov/echo/dfr_rest_services.get_cwa_eff_compliance?output=JSON&p_id=110002049239
Couldn't get this facility's info!
https://ofmpub.epa.gov/echo/dfr_rest_services.get_cwa_eff_compliance?output=JSON&p_id=110000416871
110000416871: pH
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Com

https://ofmpub.epa.gov/echo/dfr_rest_services.get_cwa_eff_compliance?output=JSON&p_id=110002050119
Couldn't get this facility's info!
https://ofmpub.epa.gov/echo/dfr_rest_services.get_cwa_eff_compliance?output=JSON&p_id=110023013228
Couldn't get this facility's info!
https://ofmpub.epa.gov/echo/dfr_rest_services.get_cwa_eff_compliance?output=JSON&p_id=110056339886
Couldn't get this facility's info!
https://ofmpub.epa.gov/echo/dfr_rest_services.get_cwa_eff_compliance?output=JSON&p_id=110039962644
110039962644: Chloride [as Cl]
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
Violation: 3% above limit
Violation: 4% above limit
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
Violation: 1% above limit
in Compliance
in Compliance
in Compliance
in Compliance
in Complianc

...

https://ofmpub.epa.gov/echo/dfr_rest_services.get_cwa_eff_compliance?output=JSON&p_id=110005490762
110005490762: BOD, 5-day, 20 deg. C
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
https://ofmpub.epa.gov/echo/dfr_rest_services.get_cwa_eff_compliance?output=JSON&p_id=110002321210
110002321210: Phosphorus, total [as P]
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
i

https://ofmpub.epa.gov/echo/dfr_rest_services.get_cwa_eff_compliance?output=JSON&p_id=110039961556
110039961556: BOD, 5-day, 20 deg. C
Violation: 10% above limit
in Compliance
Violation: 8% above limit
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
Violation: 22% above limit
Violation: 29% above limit
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
Violation: 4% above limit
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
VCVCCCCCCCCCCVVCCCCCCCCCCCVCCCCCCCCCCCC
110039961556: Mercury, total recoverable
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Comp

https://ofmpub.epa.gov/echo/dfr_rest_services.get_cwa_eff_compliance?output=JSON&p_id=110000555097
Couldn't get this facility's info!
https://ofmpub.epa.gov/echo/dfr_rest_services.get_cwa_eff_compliance?output=JSON&p_id=110000421106
Couldn't get this facility's info!
https://ofmpub.epa.gov/echo/dfr_rest_services.get_cwa_eff_compliance?output=JSON&p_id=110010886997
110010886997: Chlorine, total residual
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
Violation: 163% above limit
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
CCCCCCCCCCCCCCCCCCCCCCCCCCCCVCCCCC

https://ofmpub.epa.gov/echo/dfr_rest_services.get_cwa_eff_compliance?output=JSON&p_id=110000422383
110000422383: BOD, 5-day, 20 deg. C
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
Violation: 42% above limit
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
CCCCCCCCCCCCCCCVCCCCCCCCCCCCCCCCCCCCCCC
110000422383: Solids, total suspended
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
in Compliance
Violation: 27% above limit
Viola

https://ofmpub.epa.gov/echo/dfr_rest_services.get_cwa_eff_compliance?output=JSON&p_id=110002035020
Couldn't get this facility's info!
https://ofmpub.epa.gov/echo/dfr_rest_services.get_cwa_eff_compliance?output=JSON&p_id=110000702473
Couldn't get this facility's info!
https://ofmpub.epa.gov/echo/dfr_rest_services.get_cwa_eff_compliance?output=JSON&p_id=110000419538
Couldn't get this facility's info!
https://ofmpub.epa.gov/echo/dfr_rest_services.get_cwa_eff_compliance?output=JSON&p_id=110002044831
Couldn't get this facility's info!
https://ofmpub.epa.gov/echo/dfr_rest_services.get_cwa_eff_compliance?output=JSON&p_id=110000422846
Couldn't get this facility's info!
DONE!


#### Which, if any, facilities appear to trying to "game" EPA's compliance algorithm, by modifying discharges to remain under the radar?

In [43]:
print(flagged)

[{'fac': '110000422392', 'pollutant': 'Phosphorus, total [as P]', 'range': [27, 33]}, {'fac': '110000422392', 'pollutant': 'Phosphorus, total [as P]', 'range': [30, 36]}, {'fac': '110013863275', 'pollutant': 'Copper, total recoverable', 'range': [3, 9]}, {'fac': '110039961397', 'pollutant': 'BOD, 5-day, 20 deg. C', 'range': [9, 15]}, {'fac': '110039961397', 'pollutant': 'BOD, 5-day, 20 deg. C', 'range': [24, 30]}, {'fac': '110002050182', 'pollutant': 'Chloride [as Cl]', 'range': [9, 15]}, {'fac': '110000736124', 'pollutant': 'Solids, total suspended', 'range': [3, 9]}, {'fac': '110039975827', 'pollutant': 'BOD, 5-day, 20 deg. C', 'range': [15, 21]}, {'fac': '110002321176', 'pollutant': 'Chloride [as Cl]', 'range': [9, 15]}, {'fac': '110002321176', 'pollutant': 'Chloride [as Cl]', 'range': [12, 18]}, {'fac': '110039961556', 'pollutant': 'Oxygen, dissolved [DO]', 'range': [27, 33]}, {'fac': '110039961556', 'pollutant': 'Oxygen, dissolved [DO]', 'range': [30, 36]}, {'fac': '110027215500',